## Fine-tune large models using 🤗 `peft` adapters, `transformers` & `bitsandbytes`

In this tutorial we will cover how we can fine-tune large language models using the very recent `peft` library and `bitsandbytes` for loading large models in 8-bit.
The fine-tuning method will rely on a recent method called "Low Rank Adapters" (LoRA), instead of fine-tuning the entire model you just have to fine-tune these adapters and load them properly inside the model.
After fine-tuning the model you can also share your adapters on the 🤗 Hub and load them very easily. Let's get started!

### Install requirements

First, run the cells below to install the requirements:

In [1]:
# !pip install -q bitsandbytes datasets accelerate loralib
# !pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

### Model loading

Here let's load the `opt-6.7b` model, its weights in half-precision (float16) are about 13GB on the Hub! If we load them in 8-bit we would require around 7GB of memory instead.

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(
    "stas/tiny-random-llama-2",
    quantization_config=config,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("stas/tiny-random-llama-2")
tokenizer.pad_token = tokenizer.eos_token

D:\Clifton\Dokumente\Uni\nlp\selfcontained-adapters-dev\hf_transformers\src\transformers\utils\hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


### Post-processing on the model

Finally, we need to apply some post-processing on the 8-bit model to enable training, let's freeze all our layers, and cast the layer-norm in `float32` for stability. We also cast the output of the last layer in `float32` for the same reasons.

In [3]:
for param in model.parameters():
  # param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

# model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

# Hack to prevent HF Trainer from throwing an error due to peft missing.
model._hf_peft_config_loaded = True

In [4]:
for n, p in model.named_parameters():
    print(n, p.dtype, p.device)

model.embed_tokens.weight torch.float16 cuda:0
model.layers.0.self_attn.q_proj.weight torch.uint8 cuda:0
model.layers.0.self_attn.k_proj.weight torch.uint8 cuda:0
model.layers.0.self_attn.v_proj.weight torch.uint8 cuda:0
model.layers.0.self_attn.o_proj.weight torch.uint8 cuda:0
model.layers.0.mlp.gate_proj.weight torch.uint8 cuda:0
model.layers.0.mlp.up_proj.weight torch.uint8 cuda:0
model.layers.0.mlp.down_proj.weight torch.uint8 cuda:0
model.layers.0.input_layernorm.weight torch.float32 cuda:0
model.layers.0.post_attention_layernorm.weight torch.float32 cuda:0
model.layers.1.self_attn.q_proj.weight torch.uint8 cuda:0
model.layers.1.self_attn.k_proj.weight torch.uint8 cuda:0
model.layers.1.self_attn.v_proj.weight torch.uint8 cuda:0
model.layers.1.self_attn.o_proj.weight torch.uint8 cuda:0
model.layers.1.mlp.gate_proj.weight torch.uint8 cuda:0
model.layers.1.mlp.up_proj.weight torch.uint8 cuda:0
model.layers.1.mlp.down_proj.weight torch.uint8 cuda:0
model.layers.1.input_layernorm.weigh

### Apply LoRA

Here comes the magic with `peft`! Let's load a `PeftModel` and specify that we are going to use low-rank adapters (LoRA) using `get_peft_model` utility function from `peft`.

In [5]:
# from peft import LoraConfig, get_peft_model

# config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     # target_modules="all-linear",
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM"
# )

# model = get_peft_model(model, config)
# model.print_trainable_parameters()

In [6]:
# type(model.base_model.h[0].attn.c_attn)

In [7]:
import adapters
from adapters import LoRAConfig

adapters.init(model)

config = LoRAConfig(alpha=32, r=16, dropout=0.05)
model.add_adapter("my_adapter", config=config)
model.train_adapter("my_adapter")

print(model.adapter_summary())

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
my_adapter               lora                  2,048       3.925       1       1
--------------------------------------------------------------------------------
Full model                                    52,176     100.000               0


In [8]:
# Verifying the datatypes.
dtypes = {}
for _, p in model.named_parameters():
    dtype = p.dtype
    if dtype not in dtypes:
        dtypes[dtype] = 0
    dtypes[dtype] += p.numel()
total = 0
for k, v in dtypes.items():
    total += v
for k, v in dtypes.items():
    print(k, v, v / total)

torch.float16 96000 0.9391141023634372
torch.uint8 4096 0.04006886836750665
torch.float32 2128 0.02081702926905619


### Training

In [9]:
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)

Found cached dataset json (D:/Clifton/.torch/huggingface/datasets/Abirate___json/Abirate--english_quotes-6e72855d06356857/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at D:\Clifton\.torch\huggingface\datasets\Abirate___json\Abirate--english_quotes-6e72855d06356857\0.0.0\e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4\cache-2e9f1b08d2fb61e7.arrow


In [10]:
import transformers
from adapters import AdapterTrainer

trainer = AdapterTrainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        warmup_steps=10,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=50,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

  0%|          | 0/200 [00:00<?, ?it/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 8.024, 'learning_rate': 0.00015789473684210527, 'epoch': 0.32}
{'loss': 8.0127, 'learning_rate': 0.00010526315789473685, 'epoch': 0.64}
{'loss': 7.9989, 'learning_rate': 5.2631578947368424e-05, 'epoch': 0.96}
{'loss': 7.9917, 'learning_rate': 0.0, 'epoch': 1.27}
{'train_runtime': 49.581, 'train_samples_per_second': 64.541, 'train_steps_per_second': 4.034, 'train_loss': 8.006820220947265, 'epoch': 1.27}


TrainOutput(global_step=200, training_loss=8.006820220947265, metrics={'train_runtime': 49.581, 'train_samples_per_second': 64.541, 'train_steps_per_second': 4.034, 'train_loss': 8.006820220947265, 'epoch': 1.27})

## Share adapters on the 🤗 Hub

In [11]:
# from huggingface_hub import notebook_login

# notebook_login()

In [12]:
# model.push_to_hub("ybelkada/opt-6.7b-lora", use_auth_token=True)

## Load adapters from the Hub

You can also directly load adapters from the Hub using the commands below:

In [13]:
# import torch
# from peft import PeftModel, PeftConfig
# from transformers import AutoModelForCausalLM, AutoTokenizer

# peft_model_id = "ybelkada/opt-6.7b-lora"
# config = PeftConfig.from_pretrained(peft_model_id)
# model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# # Load the Lora model
# model = PeftModel.from_pretrained(model, peft_model_id)

## Inference

You can then directly use the trained model or the model that you have loaded from the 🤗 Hub for inference as you would do it usually in `transformers`.

In [14]:
batch = tokenizer("Two things are infinite: ", return_tensors='pt')

model.eval()
with torch.cuda.amp.autocast(enabled=False):
    print(torch.is_autocast_enabled())
    output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

D:\Clifton\Dokumente\Uni\nlp\selfcontained-adapters-dev\hf_transformers\src\transformers\generation\utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


False


D:\Clifton\Dokumente\Uni\nlp\selfcontained-adapters-dev\hf_transformers\src\transformers\generation\utils.py:1636: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(




 ▁Two▁things▁are▁infinite:▁ color meniod den local la enc preтель meniodMod redree veryiste width Niresvert partic do': Jah color meniodMod redree veryiste width Niresvert partic� :man correctblem op param They Meтельener? spe


As you can see by fine-tuning for few steps we have almost recovered the quote from Albert Einstein that is present in the [training data](https://huggingface.co/datasets/Abirate/english_quotes).